In [1]:

import mediapipe as mp
import numpy as np
import time
import cv2
import tensorflow as tf

In [2]:
"""
이거 쓰세용

"""
keypoints = []
left_hands = []
mp_holistic = mp.solutions.holistic
video_hol = mp.solutions.holistic.Holistic(static_image_mode=False,upper_body_only=False,smooth_landmarks=True,
                                           min_detection_confidence=0.5,min_tracking_confidence=0.5)
# Prepare DrawingSpec for drawing the face landmarks later.
mp_drawing = mp.solutions.drawing_utils 
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)

# For webcam input:
holistic = mp_holistic.Holistic(
    min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv2.VideoCapture("./video/KETI_SL_0000006020.avi")

cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

# cap.set(cv2.CAP_PROP_FPS, 1)

total_frames = cap.get(7)
print(total_frames)

print(cap.get(cv2.CAP_PROP_FPS))

frame_rate = 10
prev = 0
i = 0
left_hand_lists = []
right_hand_lists = []
face_lists = []
pose_lists = []


while cap.isOpened():
#     time_elapsed = time.time() - prev
    success, image = cap.read()

#     if time_elapsed > 1./frame_rate:
#         prev = time.time()
        
    if not success:
        print("Ignoring empty camera frame.")
    # If loading a video, use 'break' instead of 'continue'.
        break

    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = holistic.process(image)

    # Draw landmark annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
      image, results.face_landmarks, mp_holistic.FACE_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(
      image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

    if results.left_hand_landmarks:
        left_hand_list = []
        for lm in results.left_hand_landmarks.landmark:
            left_hand_list.append([lm.x, lm.y, lm.z])
        left_hand_lists.append(left_hand_list)
        
    if results.right_hand_landmarks:
        right_hand_list = []
        for lm in results.right_hand_landmarks.landmark:
            right_hand_list.append([lm.x, lm.y, lm.z])
        right_hand_lists.append(right_hand_list)
        
    if results.face_landmarks:
        face_list = []
        for lm in results.face_landmarks.landmark:
            face_list.append([lm.x, lm.y, lm.z])
        face_lists.append(face_list)
        
    if results.pose_landmarks:
        pose_list = []
        for lm in results.pose_landmarks.landmark:
            pose_list.append([lm.x, lm.y, lm.z])
        pose_lists.append(pose_list)
        
        
    cv2.imshow('MediaPipe Holistic', image)
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break
holistic.close()
cap.release()
cv2.destroyAllWindows()

95.0
30.0
Ignoring empty camera frame.


In [3]:
import torch
lt = torch.FloatTensor(left_hand_lists)
rt = torch.FloatTensor(right_hand_lists)
ft = torch.FloatTensor(face_lists)
pt = torch.FloatTensor(pose_lists)


print(
torch.FloatTensor(left_hand_lists).shape,
torch.FloatTensor(right_hand_lists).shape,
torch.FloatTensor(face_lists).shape,
torch.FloatTensor(pose_lists).shape)

torch.Size([72, 21, 3]) torch.Size([3, 21, 3]) torch.Size([95, 468, 3]) torch.Size([95, 33, 3])


In [4]:
torch.save(lt, 'tensor_lt.pt')
torch.save(rt, 'tensor_rt.pt')
torch.save(ft, 'tensor_ft.pt')
torch.save(pt, 'tensor_pt.pt')

In [5]:
print(torch.load('tensor.pt'))

tensor([[[ 6.1675e-01,  8.5547e-01, -2.7421e-05],
         [ 5.8203e-01,  8.7212e-01, -4.2035e-03],
         [ 5.5581e-01,  9.0538e-01, -1.7339e-02],
         ...,
         [ 6.2943e-01,  9.8285e-01,  1.9103e-05],
         [ 6.2487e-01,  9.6758e-01,  5.2582e-03],
         [ 6.2492e-01,  9.5232e-01,  2.4131e-03]],

        [[ 6.2189e-01,  8.0944e-01, -1.3018e-05],
         [ 5.9213e-01,  8.1036e-01, -2.1950e-02],
         [ 5.6735e-01,  8.4304e-01, -4.1754e-02],
         ...,
         [ 5.7168e-01,  9.2639e-01, -3.4545e-02],
         [ 5.7442e-01,  9.1128e-01, -3.0186e-02],
         [ 5.8026e-01,  8.9252e-01, -3.3501e-02]],

        [[ 5.9675e-01,  6.9303e-01, -3.7935e-05],
         [ 5.7199e-01,  6.4857e-01, -1.1915e-02],
         [ 5.4409e-01,  6.3608e-01, -2.8128e-02],
         ...,
         [ 5.5716e-01,  7.7775e-01, -3.5100e-02],
         [ 5.5943e-01,  7.6213e-01, -3.0627e-02],
         [ 5.6655e-01,  7.5363e-01, -3.1632e-02]],

        ...,

        [[ 6.1158e-01,  8.3502e-01,  8